In [5]:
import glob
import pandas as pd
import os
import json

import warnings
warnings.filterwarnings('ignore')

# Daily Data

In [6]:
# inputdir = "../../data/website/databricks/hourly"
# outputdir = "../../data/website/test/hourly"

# problematicpaths = []

# for tt in ["user", "spider"]:
#     for at in ["desktop", "mobile-dev", "mobile-app"]:
#         for domain in ["en.wikipedia", "de.wikipedia", "fr.wikipedia", "es.wikipedia", "ru.wikipedia", "zh.wikipedia"]:
#             for year in [2019]:
#                 for month in [9]:
#                     for day in range(1,32):

#                         jsondic = {i:[] for i in range(24)}
#                         filepathpattern = f"./{inputdir}/{tt}/{at}/{domain}/{year}-{month:02d}-{day:02d}-*.csv"
#                         filepaths = glob.glob(filepathpattern)
                        
#                         if not filepaths:
#                             print(f"{filepathpattern} IS EMPTY")
#                             problematicpaths.append(filepathpattern)
#                             continue

#                         elif len(filepaths) != 24:
#                             print(f"{filepathpattern} DOES NOT CONTAIN 24 HOURS")
#                             problematicpaths.append(filepathpattern)
#                             continue
                        
#                         for filepath in filepaths:
#                             hour = int(filepath.split("-")[-1].split(".")[0])
#                             nested_csvs = glob.glob(filepath + "/*.csv")
#                             if len(nested_csvs) != 1:
#                                 print(f"{filepathpattern} CONTAINS NOT EXACTLY 1 NESTED CSV")
#                                 break
#                             realfilepath = nested_csvs[0]
#                             with open(realfilepath, "r") as f:
#                                 for line in f.read().split("\n"):
#                                     x, y = line.split(",")
#                                     jsondic[hour].append({"x":int(x), "y":int(y)})

#                         outfilepath = f"./{outputdir}/{tt}/{at}/{domain}/{year}-{month:02d}-{day:02d}.json"
#                         outfiledir = "/".join(outfilepath.split("/")[:-1])

#                         if not os.path.exists(outfiledir):
#                             os.makedirs(outfiledir)

#                         with open(outfilepath, "w") as f:
#                             json.dump(jsondic, f)

In [21]:
# !dbfs cp -r "dbfs:/mnt/group09/websitedata123/nice.parquet" "../../data/website/databricks/hourly2/sep2019.parquet"
# !dbfs cp -r "dbfs:/mnt/group09/websitedata123/sep2018.parquet" "../../data/website/databricks/hourly2/sep2018.parquet"
# !dbfs cp -r "dbfs:/mnt/group09/websitedata123/oct2019.parquet" "../../data/website/databricks/hourly2/oct2019.parquet"

dbfs:/mnt/group09/websitedata123/oct2019.parquet/_SUCCESS -> ../../data/website/databricks/hourly2/oct2019.parquet/_SUCCESS
dbfs:/mnt/group09/websitedata123/oct2019.parquet/_committed_6403748976473738617 -> ../../data/website/databricks/hourly2/oct2019.parquet/_committed_6403748976473738617
dbfs:/mnt/group09/websitedata123/oct2019.parquet/_started_6403748976473738617 -> ../../data/website/databricks/hourly2/oct2019.parquet/_started_6403748976473738617
dbfs:/mnt/group09/websitedata123/oct2019.parquet/part-00000-tid-6403748976473738617-74b0a401-c1f1-48b1-80ff-40dee974c842-3083605-1.c000.snappy.parquet -> ../../data/website/databricks/hourly2/oct2019.parquet/part-00000-tid-6403748976473738617-74b0a401-c1f1-48b1-80ff-40dee974c842-3083605-1.c000.snappy.parquet
dbfs:/mnt/group09/websitedata123/oct2019.parquet/part-00001-tid-6403748976473738617-74b0a401-c1f1-48b1-80ff-40dee974c842-3083597-1.c000.snappy.parquet -> ../../data/website/databricks/hourly2/oct2019.parquet/part-00001-tid-64037489764

In [23]:
df = pd.read_parquet("../../data/website/databricks/hourly2/oct2019.parquet").drop_duplicates()
df["date"] = pd.to_datetime(df["timestamp"], unit="s")
df["year"] = df.date.dt.year
df["month"] = df.date.dt.month
df["day"] = df.date.dt.day
df["hour"] = df.date.dt.hour
df

,timestamp,domain,trafficType,accessType,y,x,date,year,month,day,hour
0,1570525200,en.wikipedia,spider,desktop,125,1,2019-10-08 09:00:00,2019,10,8,9
1,1570478400,de.wikipedia,spider,desktop,2,121,2019-10-07 20:00:00,2019,10,7,20
2,1570471200,de.wikipedia,user,mobile-app,29,8,2019-10-07 18:00:00,2019,10,7,18
3,1570474800,de.wikipedia,user,mobile-app,58,2,2019-10-07 19:00:00,2019,10,7,19
4,1570467600,de.wikipedia,user,desktop,5,16,2019-10-07 17:00:00,2019,10,7,17
...,...,...,...,...,...,...,...,...,...,...,...
188332,1570154400,ru.wikipedia,user,mobile-web,9,1,2019-10-04 02:00:00,2019,10,4,2
188333,1570179600,en.wikipedia,user,desktop,36,4,2019-10-04 09:00:00,2019,10,4,9
188334,1570136400,de.wikipedia,user,mobile-app,562,1,2019-10-03 21:00:00,2019,10,3,21
188335,1570140000,en.wikipedia,user,mobile-web,13596,1,2019-10-03 22:00:00,2019,10,3,22


In [24]:
def df_to_json(df):
    outputdir = "../../data/website/hourly"

    jsondic = {i:[] for i in range(24)}

    # for hour in range(24):
    #     # cumx = 0
    #     for _, (x, y) in df[df["hour"]==hour][["x", "y"]].iterrows():
    #         # cumx += x
    #         jsondic[hour].append({"x":int(cumx), "y":int(y)})

    for _, (hour, x, y) in df[["hour", "x", "y"]].iterrows():
        jsondic[hour].append({"x":int(x), "y":int(y)})

    tt = df["trafficType"].iloc[0]
    at = df["accessType"].iloc[0]
    domain = df["domain"].iloc[0]
    year = df["year"].iloc[0]
    month = df["month"].iloc[0]
    day = df["day"].iloc[0]

    outfilepath = f"./{outputdir}/{tt}/{at}/{domain}/{year}-{month:02d}-{day:02d}.json"
    outfiledir = "/".join(outfilepath.split("/")[:-1])

    if not os.path.exists(outfiledir):
        os.makedirs(outfiledir)

    with open(outfilepath, "w") as f:
        json.dump(jsondic, f)

In [25]:
df\
.sort_values(["trafficType", "accessType", "domain", "year", "month", "day", "hour", "y"], ascending=[False, False, False, True, True, True, True, False])\
.groupby(["trafficType", "accessType", "domain", "year", "month", "day"])\
.apply(df_to_json)

""


In [191]:
# outputdir = "../../data/website/hourly"

# problematicpaths = []

# prefilter_df = df
# for tt in ["spider"]:#["user", "spider"]:
#     # prefilter_df = prefilter_df[(df["trafficType"] == tt)]
#     for at in ["mobile-web"]:#["desktop", "mobile-dev", "mobile-app"]:
#         # prefilter_df = prefilter_df[(df["accessType"] == at)]
#         for domain in ["en.wikipedia", "de.wikipedia", "fr.wikipedia", "es.wikipedia", "ru.wikipedia", "zh.wikipedia"]:
#             # prefilter_df = prefilter_df[(df["domain"] == domain)]
#             for year in [2019]:
#                 # prefilter_df = prefilter_df[(df["date"].dt.year == year)]
#                 for month in [9]:
#                     # prefilter_df = prefilter_df[(df["date"].dt.month == month)]
#                     for day in range(1,32):
#                         # prefilter_df = prefilter_df[(df["date"].dt.day == day)]

#                         jsondic = {i:[] for i in range(24)}

#                         # print(1)
#                         # for hour in range(24):

#                         #     # print(1)
#                         #     filtered_df = prefilter_df[
#                         #         (df["date"].dt.hour == hour)
#                         #     ]

#                         #     # filtered_df.sort_values("y", ascending=False, inplace=True)

#                         #     # print(2)
#                         #     cumx = 0
#                         #     for _, (x, y) in filtered_df[["x", "y"]].iterrows():
#                         #         cumx += x
#                         #         jsondic[hour].append({"x":int(cumx), "y":int(y)})

#                         # print(3)
#                         outfilepath = f"./{outputdir}/{tt}/{at}/{domain}/{year}-{month:02d}-{day:02d}.json"
#                         outfiledir = "/".join(outfilepath.split("/")[:-1])

#                         # print(4)
#                         if not os.path.exists(outfiledir):
#                             os.makedirs(outfiledir)

#                     #     print(5)
#                         with open(outfilepath, "w") as f:
#                             json.dump(jsondic, f)

#                     # 8/0

# Monthly Data

In [27]:
# !dbfs cp -r "dbfs:/mnt/group09/websitedatamonthly/sep2018.parquet" "../../data/website/databricks/monthly/sep2018.parquet"
# !dbfs cp -r "dbfs:/mnt/group09/websitedatamonthly/sep2019.parquet" "../../data/website/databricks/monthly/sep2019.parquet"
!dbfs cp -r "dbfs:/mnt/group09/websitedatamonthly/oct2019.parquet" "../../data/website/databricks/monthly/oct2019.parquet"

dbfs:/mnt/group09/websitedatamonthly/oct2019.parquet/_SUCCESS -> ../../data/website/databricks/monthly/oct2019.parquet/_SUCCESS
dbfs:/mnt/group09/websitedatamonthly/oct2019.parquet/_committed_3991830075715639578 -> ../../data/website/databricks/monthly/oct2019.parquet/_committed_3991830075715639578
dbfs:/mnt/group09/websitedatamonthly/oct2019.parquet/_started_3991830075715639578 -> ../../data/website/databricks/monthly/oct2019.parquet/_started_3991830075715639578
dbfs:/mnt/group09/websitedatamonthly/oct2019.parquet/part-00000-tid-3991830075715639578-6cb3ee07-9699-4a90-896d-3da10f0e6309-3078931-1.c000.snappy.parquet -> ../../data/website/databricks/monthly/oct2019.parquet/part-00000-tid-3991830075715639578-6cb3ee07-9699-4a90-896d-3da10f0e6309-3078931-1.c000.snappy.parquet
dbfs:/mnt/group09/websitedatamonthly/oct2019.parquet/part-00001-tid-3991830075715639578-6cb3ee07-9699-4a90-896d-3da10f0e6309-3078933-1.c000.snappy.parquet -> ../../data/website/databricks/monthly/oct2019.parquet/part-0

In [28]:
df = pd.read_parquet("../../data/website/databricks/monthly/oct2019.parquet").drop_duplicates()
df["date"] = pd.to_datetime(df["timestamp"], unit="s")
df

,timestamp,domain,trafficType,accessType,sumcount,date
0,1570557600,es.wikipedia,user,mobile-web,34494,2019-10-08 18:00:00
1,1570471200,de.wikipedia,user,mobile-app,75817,2019-10-07 18:00:00
2,1570597200,es.wikipedia,spider,desktop,3096,2019-10-09 05:00:00
3,1570028400,de.wikipedia,spider,desktop,31569,2019-10-02 15:00:00
4,1570489200,en.wikipedia,user,mobile-app,98549,2019-10-07 23:00:00
...,...,...,...,...,...,...
8635,1570143600,es.wikipedia,spider,desktop,10873,2019-10-03 23:00:00
8636,1570168800,en.wikipedia,spider,desktop,314435,2019-10-04 06:00:00
8637,1570575600,fr.wikipedia,user,mobile-app,5977,2019-10-08 23:00:00
8638,1570392000,ru.wikipedia,user,mobile-web,6624,2019-10-06 20:00:00


In [29]:
outputdir = "../../data/website/monthly"

problematicpaths = []

for tt in ["user", "spider"]:
    for at in ["desktop", "mobile-web", "mobile-app"]:
        for domain in ["en.wikipedia", "de.wikipedia", "fr.wikipedia", "es.wikipedia", "ru.wikipedia", "zh.wikipedia"]:
            for year in [2019]:
                for month in [10]:

                    jsondic = []

                    filtered_df = df[
                        (df["domain"] == domain) &
                        (df["accessType"] == at) &
                        (df["trafficType"] == tt) &
                        (df["date"].dt.year == year) &
                        (df["date"].dt.month == month)
                    ]

                    filtered_df.sort_values("timestamp", ascending=True, inplace=True)

                    for _, (date, y) in filtered_df[["date", "sumcount"]].iterrows():
                        x = f"{date.year:04d}-{date.month:02d}-{date.day:02d}-{date.hour:02d}"
                        jsondic.append({"x":x, "y":int(y)})

                    outfilepath = f"./{outputdir}/{tt}/{at}/{domain}/{year}-{month:02d}.json"
                    outfiledir = "/".join(outfilepath.split("/")[:-1])  # the directory where the file is in

                    if not os.path.exists(outfiledir):
                        os.makedirs(outfiledir)

                    with open(outfilepath, "w") as f:
                        json.dump(jsondic, f)

In [50]:
# !rm ../../data/website/hourly/*/*/*/2020-*.json

zsh:1: no matches found: ../../data/website/hourly/*/*/*/2020-*.json
